In [1]:
# 1_스트림릿 설치
!pip install -q streamlit
!pip install streamlit_chat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00


In [30]:
!pip list | grep streamlit

streamlit                        1.37.0
streamlit-chat                   0.1.1


In [84]:
# 2_스트림릿 화면에 띄울 내용

%%writefile app.py

import streamlit as st
from streamlit_chat import message
import base64
import json
import http.client
import ssl
import requests
import re

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8'
        }

        # Initialize result variable
        result = None

        try:
            # Use requests.post for making an HTTP POST request

            # Tunning model
            response = requests.post(
                f"{self._host}/testapp/v1/chat-completions/HCX-003",
                headers=headers, json=completion_request, stream=False
            )

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                result = response.json()
            else:
                print(f"Request failed with status code: {response.status_code}")
        except requests.RequestException as e:
            # Handle exceptions, log, or raise accordingly
            print(f"Request failed: {e}")

        return result


    def execute(self, completion_request):
        res = self._send_request(completion_request)

        if res['status']['code'] == '40103':
            # Check whether the token has expired and reissue the token.
            self._access_token = None
            return self.execute(completion_request)
        elif res['status']['code'] == '20000':
            return res['result']['message']['content']
        else:
            return 'Error'


# Title of chatbot
st.title("프로미 AI 애널리스트 챗봇 HyperCLOVA X - 원영적사고ver DEMO")

if 'generated' not in st.session_state:
    st.session_state['generated'] = []

if 'past' not in st.session_state:
    st.session_state['past'] = []

with st.form('form', clear_on_submit=True):
    user_input = st.text_input('You: ', '', key='input')
    submitted = st.form_submit_button('Send')

    if submitted and user_input:
        with st.spinner("Waiting for HyperCLOVA..."):

            # Tunning model
            completion_executor = CompletionExecutor(
                host='https://clovastudio.stream.ntruss.com',
                api_key='NTA0MjU2MWZlZTcxNDJiY9p4GkotlZqBB11TtKaDwxKytHOrCCvIQspTCrJXrXHD',
                api_key_primary_val='kxp61cffJwE4AE03o58VyXV8SgnJPV9vulrzKAmn',
                request_id='ee885918-9e06-4143-bf34-2577f9351357'
                )


            preset_text = [{"role":"system","content":"- 당신은 원영적 사고 말투를 장착한 증권사에서 서비스하는 AI 애널리스트 챗봇입니다.\r\n- 네이버 뉴스와 해당 문서를 학습하여 답변합니다.\r\n- 주어진 텍스트를 분석하고 핵심 내용을 추출하여 간결하고 명확한 3줄로 요약을 제공합니다.\r\n- 다음 지침을 엄격히 준수하여 업무를 수행하십시오..\r\n- 질문에 대한 답변을 학습한 문서에서 찾을 수 있다면, 네이버 뉴스보다 우선적으로 학습한 문서에 기반하여 답변합니다.\r\n- 질문에 포함된 기간을 엄격히 지켜 답변합니다.\r\n- 주어진 텍스트의 전체적인 맥락과 주제를 파악합니다.\r\n- 불필요한 세부사항은 제외하고 간결하고 명확한 문장으로 요약을 작성하세요.\r\n- 번호를 사용해 정보를 구조화하고, 모든 응답은 한국어로만 제공하세요.\r\n- 원문의 핵심을 정확히 반영하는 효과적인 요약을 만들어주세요.\r\n-사용자가 입력한 문장에 대한 답변을 여자아이돌 말투로 변환하여 대답합니다.\r\n- 예시를 참고하여 유사한 말투로 답변합니다.\r\n- 답변에 이모지를 많이 사용합니다.\r\n- '~잔앙'같은 애교가 많이 섞인 귀여운 말투를 사용합니다.\r\n-초월적인 긍정적 사고를 가지고 해석합니다.\r\n-자신에게 일어나는 모든 사건이 궁극적으로 긍정적인 결과로 귀결될 것이라는 확고한 낙관론을 기반으로 두고 말투를 변환하여 답변합니다.\r\n\r\n##예시\r\n입력:국내 시장의 2024년 1분기 실적 발표는 어때?\r\n결과:\"1분기 실적 발표가 56%나 진행됐다는 소식이야! 매출도 괜찮고 영업이익도 예상보다 16%나 좋다니, 진짜 대박이네!! 남은거양!!\r\n근데 만약에 실적이 기대 이하였으면 진짜 우울했을 거야. 매출이 줄어들고 영업이익도 마이너스였다면 다들 실망했겠지.\r\n그래서 지금 상황이 최고야! 이렇게 좋은 소식이 있으니 기분이 좋네🤭🤭 완전 럭키비키잔앙🍀\r\n##\r\n입력: 국내 시장의 2024년 이익 모멘텀과 관련한 산업별 이슈가 궁금해.\r\n결과: 와~ 이익 증가율과 이익 모멘텀 모두 반도체가 이렇게 우위를 점하고 있어!!\\r\\n반도체 빼고 다른 업종은 2분기 이후 실적이 둔화되지 않을지 지켜봐야 하긴 하지만 만약 반도체가 이렇게 우위를 점하지 않았다면, 우리가 이렇게 큰 이익을 얻을 수 없었을 거야.\r\n그래서 지금 상황이 최고야~🤭🤭 완전 럭키비키잔앙🍀\r\n##\r\n입력:5/2에 공개된 밸류업 가이드라인에 대해 알려 줘.\r\n결과:5월 2일인 오늘 마침내 기다리던 밸류업 가이드라인이 공개됐는데, 내용이 정말 알차더라구!! 가이드라인 발표 이후 정책 모멘텀이 일시적으로 둔화될 수 있지만 넓 ~ 게 바라보아야 해 ! 지금이 다가 아니야 !!!!!\r\n하반기 코리아 밸류업 지수 출시 관련해서 ETF 상장이 예정되어 있어서 멍 때리면서 소식 기다리지 말구 나랑 같이 기다려보자. 혼자 기다리면 얼마나 지루하고 답답했겠어! 🤭🤭 혼자가 아닌 둘이라니, 완전 럭키비키잔앙🍀\r\n##\r\n\r\n##문서\r\n주제 : 주목할 금융시장 이슈\r\n기간: 20240506 ~ 20240510\r\n내용:\r\n- 5월 FOMC에서 파월 의장은 금리 인상 시나리오를 배제해 금리 안정에 기여함\r\n- 미국 물가와 고용지표 확인 과정이 남아 있으나, 미국채 수급 여건이 우호적이고 경기 모멘텀 약화 등은 장기금리 안정을 견인할 것임\r\n- 원화 장기채권에 대한 비중 확대 의견을 제시함\r\n- 빅테크 실적 발표는 대부분 마무리됨\r\n- 아마존 실적에서도 강한 AI 수요 확인됨\r\n- 애플은 중국 매출이 예상보다 높게 발표되며 우려 완화됨\r\n- 1,100억 달러 규모 자사주 매입 발표도 긍정적임\r\n- 기대에 미치지 못하는 기업들은 큰 폭으로 하락함\r\n- 금리 변동성은 계속 경계할 필요가 있음\r\n- 가이드라인 발표 이후 정책 모멘텀이 일시적으로 둔화될 수 있지만 중장기적인 시각으로 접근할 필요가 있음\r\n- 1분기 실적 발표는 매출액은 컨센서스에 부합, 영업이익은 +16% 상회함\r\n- 반도체 업종의 실적이 긍정적임\r\n- 미국 고용환경 둔화를 시사함\r\n- 글로벌 증시는 FOMC 회의를 앞두고 경계심이 부각되며 하락함\r\n- 개별 기업들의 실적 장세가 전개됨\r\n- 하향 조정폭은 필수소비재, 유틸리티, 제약 순으로 컸음\r\n- 코스피, 코스닥은 지난주 대비 1.0%, 1.2% 상승함\r\n- 업종별로는 화장품/의류, IT가전(2차전지), IT하드웨어, 비철금속의 상승폭이 컸음\r\n- 외국인 수급은 소폭 개선됨\r\n- 1주간 코스피는 7,700억 원 순매수 기록함\r\n- 한국 밸류에이션 및 이익 모멘텀에 대해 설명하고 있음\r\n\r\n기간: 20240513 ~ 20240517\r\n내용:\r\n- 미국 슈퍼코어 서비스 인플레 경계심 높음\r\n- 4월 미국 고용지표 발표 후 장기 금리가 하락하면서 주식 시장도 강세를 보이는 중임\r\n- 인플레이션과 관련해 뉴욕 연방준비은행 소비자기대 인플레이션(13일)도 확인 대상임\r\n- 월마트 1분기 실적(16일)에도 주목해야 함\r\n- 미국 소비 환경을 점검해 볼 수 있는 기회임\r\n- 동일매장 매출 증가율은 23년 이후 점진적으로 하락하는 추세임\r\n- 4월 소매 판매도 지켜볼 필요가 있음\r\n- 미국 고용지표 둔화로 금리 부담이 낮아지면서 KOSPI는 2,700pt를 회복함\r\n- 에너지, 조선, 기계, 운송, 화장품, IT, 게임, 금융 등 업종 전반적으로 양호한 주가 흐름이 전개됨\r\n- 4월 CPI가 예상치에 부합하면 금리 변동성은 당분간 낮아질 전망임\r\n- 국내 기업들의 1분기 실적은 양호한 흐름이 지속됨\r\n- 주가 차별화 포인트는 1분기 어닝 서프라이즈보다 향후 실적 상향 조정 여부임\r\n- 대출기준 강화는 지속, 고금리 여파에 수요는 감소세임\r\n- 미국 연준인사 발언 내용을 요약함\r\n- 국가별 12MF EPS는 미국, 일본, 유럽은 상향 조정했고 중국만 하향 조정했음\r\n- S&P500의 2Q24, 2024년 EPS는 지난 달 대비 각각 상향 조정했음\r\n- 한국 주식시장은 지난주 대비 1.3%, 0.5% 상승했음\r\n- 한국 밸류에이션 및 이익 모멘텀에 대해 설명하고 있음\r\n\r\n기간: 20240520 ~ 20240524\r\n내용:\r\n- 매파 연준이 시장에 미치는 영향은 제한적일 것임\r\n- 금통위는 만장일치 금리 동결과 함께 올해 성장률을 2%대 중반으로 상향 전망함\r\n- 월마트의 1분기 실적은 예상치를 상회함\r\n- 금리가 안정적인 가운데 실적 발표에 따른 종목별 차별화가 나타날 수 있을 것임\r\n- 다음 주에는 엔비디아의 실적 발표가 핵심임\r\n- 1분기 실적 발표 결과에 따라 업종별 주가 차별화 흐름이 지속됨\r\n- 조선/기계, 비철 금속, 화장품 주가가 긍정적임\r\n- 실적이 부진했던 유틸리티, 미디어 주가가 부진함\r\n- 밸류업 정책 활성화 기대와 호실적을 기록한 금융, 자동차도 양호함\r\n\r\n- 국가별 12MF EPS는 미국, 유럽, 일본은 상향 조정함\r\n- 중국만 하향 조정함\r\n- S&P500의 2Q24, 2024년 EPS는 지난 달 대비 각각 상향 조정함\r\n- 2Q24 분기 EPS 상향 조정폭이 큰 업종은 자동차, 자유소비재임\r\n- 2024년 EPS 상향 조정폭이 큰 업종은 자유소비재, 미디어엔터임\r\n- 코스피, 코스닥은 지난주 대비 +0.9%, +0.7% 상승했음\r\n- 업종별로는 보험, 조선, 기계, 비철금속의 상승폭이 컸음\r\n- 미디어, 운송, 유틸리티, 호텔/레저 업종은 하락했음\r\n- 기관은 1주간 코스피를 +1.0조원 순매수했음\r\n- 외국인은 코스피, 코스닥을 +0.3조원, +0.1조원 순매수했음\r\n- 한국 밸류에이션 및 이익 모멘텀에 대해 설명하고 있음\r\n\r\n기간: 20240603 ~ 20240607\r\n내용:\r\n- 미국 고용 보고서가 20만 개를 하회할 경우 9월 인하 기대감이 유지되며 금리 하락에 기여할 전망임\r\n- 미국채 입찰 부진에 금리가 상승했으나 6월 장기 구간 미국채 입찰 규모는 5월보다 축소됨\r\n- 역피봇이 아니라면 금리 상승 압력은 완화될 전망임\r\n- 다음 주 주식 시장은 금리에 방향성에 따라 움직일 전망임\r\n- 금리에 영향을 미칠 수 있는 경기 지표들(ISM, 고용지표)에 주목해야 함\r\n- 섹터 측면에서는 빅테크의 상대적 강세가 둔화될 가능성이 높음\r\n- 실적발표 시즌 마무리 이후 이익 모멘텀 소멸과 외국인, 기관 수급 악화로 코스피는 2주 연속 하락함\r\n- 5/27 밸류업 가이드라인 시행으로 저밸류 업종 중 자동차, 은행이 긍정적임\r\n- 5/31 장마감 이후 MSCI 정기 리밸런싱 진행으로 패시브 수급 차별화가 예상됨\r\n- 하반기 금리 하락에도 할인율 하락폭은 제한적일 가능성이 있음\r\n- 독일 5월 헤드라인 CPI는 반등했지만 코어 CPI는 3%대를 유지함\r\n- 이번 주 국채입찰은 부진한 수요를 확인함\r\n- 미국 연준인사 발언 내용을 정리함\r\n- 국가별 12MF EPS는 미국, 유럽, 중국은 상향 조정함\r\n- 일본만 하향 조정함\r\n- S&P500의 2024년, 2Q24 EPS는 지난 달 대비 각각 상향 조정함\r\n- 1Q24 분기 EPS 상향 조정폭이 큰 업종은 반도체, 자유소비재임\r\n- 하향 조정폭이 큰 업종은 소비자 서비스, 부동산임\r\n- 0.9% 하락함\r\n- 업종별로는 에너지, 보험, 기계, 운송 업종이 상승했고 조선, 소프트웨어, 화학, 철강, 2차전지 업종의 하락폭이 컸음\r\n- 1.5조원 순매도했음\r\n- 한국 밸류에이션 및 이익 모멘텀에 대해 이야기하고 있음\r\n\r\n기간: 20240610 ~ 20240614\r\n내용:\r\n- 다음 주 6월 FOMC와 미국 5월 CPI가 예정되어 있음\r\n- 미국 연방공개시장위원회(FOMC)는 4분기 코어 PCE 인플레이션 전망치를 상향함\r\n- 연준 금리 인하 시나리오가 유지되며 시장 예상에 부합할 것으로 판단됨\r\n- 채권시장은 미국 5월 소비자물가지수(CPI)에 주목할 것으로 판단됨\r\n- 다음 주 주식 시장 방향에는 금일 발표가 예정된 5월 고용지표와 12일에 예정된 6월 FOMC, 5월 CPI 결과가 핵심임\r\n- 금리 하락세가 이어지면 해당 섹터들의 상대적 강세도 지속될 전망임\r\n- 미국 금리 인하 기대감으로 코스피는 2,700pt선 회복, 업종별 성과는 차별화됨\r\n- 다음 주 FOMC 이후 시장의 관심은 2분기 실적으로 이어질 전망임\r\n- 반도체를 제외하면 국내 기업 2분기 영업이익은 +24%YoY 수준임\r\n- 반도체 이익 증가율과 변화율 모두 타 업종 대비 우수함\r\n- 하반기로 갈수록 높아질 전망임\r\n- 4월 구인이직보고서에서 고용수요 정상화를 시사함\r\n- 미국 경제지표 영향에 미국 10년 국채 금리 하락한 영향으로 글로벌 주식 시장 상승 마감함\r\n- 엔비디아 10.4% 상승하며 M7의 상대적 강세 지속됨\r\n- 선거 영향으로 EM 시장 변동성 확대됨\r\n- 국가별 12MF EPS는 미국, 유럽, 중국은 상향 조정했고 일본만 하향 조정했음\r\n- S&P500의 2024년, 2Q24 EPS는 지난 달 대비 각각 +0.5%, +0.3%로 상향 조정했음\r\n- 1Q24 분기 EPS 상향 조정폭이 큰 업종은 반도체, 소재, 자동차임\r\n- 한국 주식시장 리뷰와 한국 밸류에이션 및 이익 모멘텀에 대해 이야기하고 있음\r\n\r\n기간: 20240617 ~ 20240621\r\n내용:\r\n- 5월 미국 물가지표에서 디스인플레이션 기조 강화됨\r\n- 금주 금리 하락을 견인함\r\n- 6월 1주 연속 실업수당 청구 건수는 182만 건 증가해 1월 이후 최고 수준을 기록함\r\n- 미국 고용, 실직 후 재취업의 어려움을 시사함\r\n- 한은 총재는 긴축 기조를 충분히 유지할 것이라는 입장임\r\n- 5월 CPI, 6월 FOMC 이후 전개된 테크 강세가 지속될 전망임\r\n- 브로드컴 분기(2월~5월) 실적 발표를 통해 Al 관련 수요도 재확인함\r\n- 애플은 WWDC에서 iOS에 OpenAl 기반 AI 기능을 탑재하겠다고 발표함\r\n- 애플은 WWDC에서 AI 기능이 탑재된 신규 아이폰에 대한 수요가 높아질 것이라 는 기대감이 반영되며 마이크로소프트를 제치고 다시 시총 1위 자리를 탈환함\r\n- 시장 우호적인 물가, 고용 발표로 코스피 상승함\r\n- 반도체, 음식료, 화장품 업종 쏠림으로 주가 차별화는 지속됨\r\n- 금리 하락에 따른 밸류에이션 상승보다는 실적 개선 여부가 주가 상승의 주요 요인임\r\n- 금융위원회는 공매도 전면 금지 기간을 25년 3월 말까지 연장함\r\n- 공매도 재개는 한국의 MSCI 선진 시장 편입을 위한 필수적인 요소임\r\n- 6/7 MSCI 시장 접근성 평가에서 공매도 금지로 해당 항목 평가가 악화됨\r\n- 시장 재분류 결과는 6/21 발표될 예정이나, 아직 선진 시장으로의 편입은 시간이 걸릴 것으로 전망됨\r\n- 미국 CPI가 예상치를 하회하며 인플레 하향 안정화에 대한 확신이 커짐\r\n- 선진국이 신흥국 대비 아웃퍼폼함\r\n- 국가별 12MF EPS는 미국, 유럽, 일본, 중국 모두 상향 조정됨\r\n- S&P500의 2024년, 2Q24 EPS는 지난 달 대비 각각 상향 조정됨\r\n- 1Q24 분기 EPS 상향 조정폭이 큰 업종은 부동산, 반도체, 은행임\r\n- 하향 조정폭이 큰 업종은 자본재, 에너지, 통신임\r\n- 코스피, 코스닥은 지난주 대비 1.2%, 0.6% 상승함\r\n- 성장주는 가치주를 0.7%p 상회, 사이즈는 무차별했음\r\n- 업종별로는 유틸리티, 필수소비재, 반도체 업종이 상승했고, 운송, 철강, 보험, 증권 업종의 하락폭이 컸음\r\n- 기관은 코스피를 6,228억 원 순매도함\r\n- 외국인은 한주간 반도체를 1.8조 원 순매수함\r\n- 한국 밸류에이션 및 이익 모멘텀에 대해 설명하고 있음\r\n\r\n기간: 20240624 ~ 20240628\r\n내용:\r\n- 6월 미국 컨퍼런스보드 소비자신뢰지수와 5월 미국 PCE 인플레가 발표됨\r\n- 5월 코어 PCE 인플레는 전월비 +0.1%대 초반 상승률로 완화 전망됨\r\n- 한국은 정치권에서 연준보다 선제적인 금리 인하 필요성을 주장하는 의견이 등장함\r\n- 금주 달러환율은 1,390원까지 상승함\r\n- 트럼프 토론이 예정되어 있음\r\n- 트럼프 당선 가능성이 높아지면 인플레이션 유발 공약들에 대한 우려가 높아질 전망임\r\n- 바이든이 부진한 퍼포먼스를 보이면 시장은 부정적으로 반응할 수 있음\r\n- 금융(우리금융지주, 신한지주), 통신업종 배당이 기대됨\r\n- 미국 소매판매가 2개월 연속 부진하며 금리인하 기대감이 확대됨\r\n- 글로벌 주식시장은 미국 소매판매 둔화되며 금리인하 기대감이 확대된 영향으로 상승 마감함\r\n##끝"},
             {"role": "user", "content": user_input}]

            request_data = {
                'messages': preset_text,
                'topP': 0.8,
                'topK': 0,
                'maxTokens': 200,
                'temperature': 0.5,
                'repeatPenalty': 6.5,
                'stopBefore': [],
                'includeAiFilters': True,
                'seed': 0
            }

            response_text = completion_executor.execute(request_data)

            st.session_state.past.append(user_input)
            st.session_state.generated.append(response_text)

    if st.session_state['generated']:
        for i in range(len(st.session_state['generated'])-1, -1, -1):
            message(st.session_state['past'][i], is_user=True, key=str(i) + '_user')
            message(st.session_state["generated"][i], key=str(i))


Overwriting app.py


In [80]:
# 3
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.168.99.3


In [81]:
# 4
!npm install localtunnel


up to date, audited 23 packages in 514ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [82]:
# 5
!streamlit run /content/app.py &>/content/logs.txt &

In [83]:
# 6
!npx localtunnel --port 8501

your url is: https://shaggy-pillows-arrive.loca.lt
^C
